## **Can "run all", just need to change all file paths first**

In [1]:
# Install required packages
!pip -q install ultralytics boxmot tqdm opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.2/282.2 kB 28.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os, csv, math, time
import cv2, torch, numpy as np
from dataclasses import dataclass
from typing import Optional, Sequence, Tuple, List
from pathlib import Path
from tqdm.notebook import tqdm
from ultralytics import YOLO
from boxmot import StrongSort
from IPython.display import Video

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
import random

def seed_all(seed=1023):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_all(2310)

In [6]:
@dataclass
class YoloCfg:
    conf: float = 0.35
    iou: float = 0.5                      # intersection over union threshold: the I:U ratio that is considered as correct
    imgsz: int = 1280
    max_det: int = 300
    classes: Optional[Sequence[int]] = None   # e.g. [0] for COCO 'person'
    agnostic_nms: bool = False
    verbose: bool = False

@dataclass
class StrongSortCfg:
    # Only applied if attributes exist on your StrongSort build (safe setattr).
    max_age: Optional[int] = 40           # keep ID alive across occlusion
    n_init: Optional[int] = 3             # frames before confirming a track
    max_iou_dist: Optional[float] = 0.7   # motion/overlap gating
    max_dist: Optional[float] = 0.2       # appearance/cosine distance gating
    nn_budget: Optional[int] = 100        # feature queue size
    half: bool = True                     # use FP16 on GPU if available
    det_thresh: float = 0.3

In [7]:
def apply_tracker_tweaks(tracker, cfg: StrongSortCfg):
    # Apply only if attribute exists in this BoxMOT version
    for k in ["max_age", "n_init", "max_iou_dist", "max_dist", "nn_budget"]:
        v = getattr(cfg, k)
        if v is not None and hasattr(tracker, k):
            setattr(tracker, k, v)
            print(f"attribute {k} set for tracker.")
        else:
            print(f"attribute {k} unsuccessfully set for tracker.")

def load_models(yolo_weights: str, reid_weights: str, ssort_cfg: StrongSortCfg):
    device = 0 if torch.cuda.is_available() else 'cpu'
    tracker = StrongSort(
        reid_weights=Path(reid_weights),
        device=device,
        half=(ssort_cfg.half and torch.cuda.is_available()),
        # BaseTracker parameters
        det_thresh=ssort_cfg.det_thresh,
        max_age=ssort_cfg.max_age,
        n_init=ssort_cfg.n_init,
        max_iou_dist=ssort_cfg.max_iou_dist,
        max_cos_dist=ssort_cfg.max_dist,
        nn_budget=ssort_cfg.nn_budget
    )

    # Full arguments for StrongSort class:
    # reid_weights: Path,
    # device: device,
    # half: bool,
    # # BaseTracker parameters
    # det_thresh: float = 0.3,
    # max_age: int = 30,
    # max_obs: int = 50,
    # min_hits: int = 3,
    # iou_threshold: float = 0.3,
    # per_class: bool = False,
    # nr_classes: int = 80,
    # asso_func: str = "iou",
    # is_obb: bool = False,
    # # StrongSort-specific parameters
    # min_conf: float = 0.1,
    # max_cos_dist: float = 0.2,
    # max_iou_dist: float = 0.7,
    # n_init: int = 3,
    # nn_budget: int = 100,
    # mc_lambda: float = 0.98,
    # ema_alpha: float = 0.9,

    print(f"StrongSort args: {vars(tracker)}")
    print(f"StrongSort Tracker args: {vars(tracker.tracker)}")
    # apply_tracker_tweaks(tracker, ssort_cfg)

    yolo = YOLO(yolo_weights)
    return yolo, tracker

def yolo_detect(yolo: YOLO, frame: np.ndarray, cfg: YoloCfg):
    r = yolo.predict(
        frame,
        conf=cfg.conf,
        iou=cfg.iou,
        imgsz=cfg.imgsz,
        max_det=cfg.max_det,
        classes=cfg.classes,
        agnostic_nms=cfg.agnostic_nms,
        verbose=cfg.verbose
    )[0]
    if r.boxes is None or r.boxes.xyxy.numel() == 0:
        return None
    boxes = r.boxes.xyxy.detach().cpu().numpy()  # [N,4]
    confs = r.boxes.conf.detach().cpu().numpy()  # [N]
    clss  = r.boxes.cls.detach().cpu().numpy()   # [N]
    dets  = np.concatenate([boxes, confs[:, None], clss[:, None]], axis=1)
    return dets  # [x1,y1,x2,y2,conf,cls]

def draw_tracks(frame: np.ndarray, tracks: np.ndarray, id_color_cache: dict):
    # tracks: Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
    for tb in tracks:
        x1, y1, x2, y2, tid, conf, cls, _ = tb
        x1,y1,x2,y2,tid = int(x1),int(y1),int(x2),int(y2),int(tid)
        color = id_color_cache.setdefault(tid, (37*tid%256, 17*tid%256, 93*tid%256))
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return frame

def export_tracks_csv(csv_path: str, rows: List[Tuple[int,int,int,int,int,int,float,int]]):
    # rows: (frame, id, x1,y1,x2,y2, conf, cls)
    if not csv_path: return
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["frame","id","x1","y1","x2","y2","conf","cls"])
        w.writerows(rows)

In [8]:
def track_video(
    src_path: str,
    dst_path: str,
    yolo: YOLO,
    tracker: StrongSort,
    yolo_cfg: YoloCfg,
    export_csv: Optional[str] = None,
    show_pbar: bool = True,
):
    cap = cv2.VideoCapture(src_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {src_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w, h = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter(dst_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or None
    print(f"🎥 Video Meta: FPS={fps:.2f}, Total Frames={total}")
    pbar = tqdm(total=total, unit="frame", disable=not show_pbar)

    csv_rows = []
    id_color_cache = {}
    frame_idx = 0

    t0 = time.time()
    while True:
        ok, frame = cap.read()
        if not ok: break

        dets = yolo_detect(yolo, frame, yolo_cfg)
        if dets is None:
            out.write(frame)
            frame_idx += 1
            pbar.update(1)
            continue

        tracks = tracker.update(dets, frame)  # -> Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
        if tracks is not None and len(tracks):
            # collect CSV rows
            for x1,y1,x2,y2,tid,conf,cls,_ in tracks:
                csv_rows.append((frame_idx, int(tid), int(x1), int(y1), int(x2), int(y2), float(conf), int(cls)))
            # draw
            frame = draw_tracks(frame, tracks, id_color_cache)

        out.write(frame)
        frame_idx += 1
        pbar.update(1)

    cap.release(); out.release()
    pbar.close()
    t1 = time.time()
    print(f"Saved video to: {dst_path} | frames={frame_idx} | time={t1-t0:.1f}s | ~{frame_idx/max(t1-t0,1):.1f} FPS")
    if export_csv:
        export_tracks_csv(export_csv, csv_rows)
        print(f"Saved track CSV to: {export_csv}")

In [16]:
"""
max_age: The maximum number of consecutive frames a track can exist without being matched to a new detection before it is deleted.
n_init: The minimum number of consecutive detections required before a track is considered "confirmed" (and ready to be output).
max_iou_dist: The maximum Intersection-over-Union (IoU) distance allowed for a match. Higher values allow a detection to be matched to a track even if it overlaps LESS.
max_dist: Higher values allow more distant detections (spatially or in appearance feature space) to be matched to a track.
nn_budget: The maximum number of appearance features (from the deep neural network) to store for each track.
half: probably to use half precision
"""
ssort_cfg = StrongSortCfg(
    max_age=30,          # keep IDs alive over occlusions/net crossings # default: 50
    n_init=12,             # ori: 3
    max_iou_dist=1,     # ori: 0.7
    max_dist=1,        # ori: 0.25
    nn_budget=240,        # ori: 120
    det_thresh=0.4,
    half=True
)

yolo_cfg = YoloCfg(
    conf=0.50,           # a bit lower to catch partial players, ori: 0.30
    iou=0.8,             # ori: 0.55
    imgsz=1280,
    max_det=2,          # ori: 200
    classes=None,        # e.g., [0] if your model uses COCO 'person'
    agnostic_nms=False,
    verbose=False
)

In [17]:
# Define model paths and input/output paths

# REID_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/REID Phua/osnet_x1_0_phua.pt"
REID_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/osnet_x1_0_badminton.pt"

YOLO_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/YOLO Phua/yolo11m_cheras.pt"
# YOLO_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/YOLO/my_yolo11m_2.pt"
YOLO_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/YOLO/my_yolo11s_finals.pt"

# INPUT_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/test/reid_test_vid_4.mp4"
# INPUT_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/CHERAS FOOTAGE/vid_2025-10-10_00-02-39.mp4"
# INPUT_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/SlowFast/01_raw/cheras_rally_2.mp4"
# INPUT_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/train/ting_12/ting_vid_12.mp4"
INPUT_VIDEO_PATH = "/content/syq_anton_rally_1.mp4"

OUTPUT_VIDEO_PATH = "/content/cheras_ann.mp4"
CSV_PATH = "/content/cheras_ann.csv"

In [18]:
yolo_model, ssort_tracker = load_models(YOLO_MODEL_PATH, REID_MODEL_PATH, ssort_cfg)

track_video(INPUT_VIDEO_PATH, OUTPUT_VIDEO_PATH, yolo_model, ssort_tracker, yolo_cfg, export_csv=CSV_PATH, show_pbar=True)

2025-10-31 15:24:00.687 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:56 | __init__ - BaseTracker initialization parameters:
2025-10-31 15:24:00.687 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:57 | __init__ - det_thresh: 0.4
2025-10-31 15:24:00.688 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:58 | __init__ - max_age: 30
2025-10-31 15:24:00.688 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:59 | __init__ - max_obs: 50
2025-10-31 15:24:00.688 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:60 | __init__ - min_hits: 3
2025-10-31 15:24:00.688 | MainProcess/MainThread | INFO     | /usr/local/lib/python3.12/dist-packages/boxmot/trackers/basetracker.py:61 | __init__ - iou_threshold: 

StrongSort args: {'det_thresh': 0.4, 'max_age': 30, 'max_obs': 50, 'min_hits': 3, 'iou_threshold': 0.3, 'per_class': False, 'nr_classes': 80, 'asso_func_name': 'iou', 'is_obb': False, 'frame_count': 0, 'active_tracks': [], 'per_class_active_tracks': None, '_first_frame_processed': False, '_first_dets_processed': False, 'last_emb_size': None, 'min_conf': 0.1, 'model': <boxmot.appearance.backends.pytorch_backend.PyTorchBackend object at 0x78be317b9790>, 'tracker': <boxmot.trackers.strongsort.sort.tracker.Tracker object at 0x78bd2c28e900>, 'cmc': <boxmot.motion.cmc.ecc.ECC object at 0x78bd2c28d490>}
StrongSort Tracker args: {'metric': <boxmot.trackers.strongsort.sort.linear_assignment.NearestNeighborDistanceMetric object at 0x78bd41830a40>, 'max_iou_dist': 1, 'max_age': 30, 'n_init': 12, '_lambda': 0, 'ema_alpha': 0.9, 'mc_lambda': 0.98, 'tracks': [], '_next_id': 1, 'cmc': <boxmot.motion.cmc.ecc.ECC object at 0x78bd2c28d730>}
🎥 Video Meta: FPS=28.99, Total Frames=572


  0%|          | 0/572 [00:00<?, ?frame/s]

Saved video to: /content/cheras_ann.mp4 | frames=572 | time=76.6s | ~7.5 FPS
Saved track CSV to: /content/cheras_ann.csv


In [29]:
!ffmpeg -i 'annotated.mp4' -vcodec libx264 -pix_fmt yuv420p -y -loglevel error 'disp.mp4'
Video('disp.mp4', embed=True, width=1080, height=720)

annotated.mp4: No such file or directory


In [ ]:
!cp /content/cheras_annotated_rally.mp4 '/content/drive/MyDrive/FIT3163,3164/YOLO Phua'

# **To rectify tracks in the generated CSV**

In [ ]:
import csv
from typing import List, Set, Dict, Tuple, Iterable

def discover_established_ids(csv_path: str, num_players: int) -> Set[str]:
    """
    First Pass: Streams the CSV to find the full set of established player IDs.
    This only stores a small set of IDs, not the whole file data.
    """
    established_ids = set()

    with open(csv_path, 'r', newline='') as file:
        reader = csv.reader(file)
        # Skip header
        try:
            next(reader)
        except StopIteration:
            return established_ids

        for line in reader:
            if len(established_ids) >= num_players:
                # Stop as soon as we've established the maximum number of IDs
                break

            player_id = line[1]
            established_ids.add(player_id)

    return established_ids

def rectify_single_frame(original_rows: List[List[str]], mapping: Dict[str, str], established_ids: Set[str]):
    """
    1.  For each ID in original_rows that is in established_ids, do not touch that row.
        Note down that that particular established_id is found present in this frame.
    2.  For each ID in original_rows that is NOT in established_ids, check the mapping dictionary.
        If the ID is present in mapping, follow the mapping and change the ID for that row.
        Note down that that particular established_id is now present in this frame.
        If the ID is NOT present in mapping, check the unfound established_ids.
        Map the unfound established_id to the current row.
        Add this mapping to the mapping dictionary.
    """
    current_frame_ids = {row[1] for row in original_rows}

    available_mapping_ids = {
        mapping[key] for key in mapping
        if key in current_frame_ids
    }

    available_est_ids = sorted([
        id for id in established_ids
        if id not in current_frame_ids
        and id not in available_mapping_ids
    ], key=int)

    for row in original_rows:
        # row[1] is the player ID
        id = row[1]

        new_row = row[:]

        # Find all the rows with established player IDs
        if id in established_ids:
            yield new_row
            continue

        if id in mapping:
            new_row[1] = mapping[id]
            yield new_row
            continue

        if len(original_rows) != len(established_ids):
            yield row
            continue

        new_row[1] = available_est_ids[0]
        mapping[id] = new_row[1]
        available_est_ids.pop()
        yield new_row

def rectify_tracks(input_csv_path: str, output_csv_path: str, established_ids: Set[str]):
    """
    Second Pass: Streams the CSV, uses a persistent map to maintain identity
    based on previous ID assignments, and writes the corrected data.
    """
    # 1. Initialization
    # sorted_established_ids = sorted(list(established_ids), key=int)

    # Persistent map: Stores 'new_id' -> 'established_id' from the first time it was seen.
    current_to_established_map: Dict[str, str] = {}

    current_frame_rows = []
    current_frame_number = None

    # Open both files for streaming
    with open(input_csv_path, 'r', newline='') as infile, \
         open(output_csv_path, 'w', newline='') as outfile:

        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # Read and write header (assuming it's the first line)
        try:
            header = next(reader)
            writer.writerow(header)
        except StopIteration:
            return

        for line in reader:
            # We must skip lines that don't have enough columns, or non-numeric IDs.
            if len(line) < 2 or not line[0].isdigit() or not line[1].isdigit():
                 # Optionally skip or write the bad line as is, but skipping is safer for logic
                 continue

            frame = line[0]
            player_id = line[1]

            if current_frame_number is None:
                current_frame_number = frame

            if frame != current_frame_number:
                # Finished a frame: process the rows and write them out
                # print(current_frame_rows)
                rectified_rows = rectify_single_frame(current_frame_rows, current_to_established_map, established_ids)
                writer.writerows(rectified_rows)

                # Reset for the new frame
                current_frame_number = frame
                current_frame_rows = []

            # Store the current line's data for processing when the frame is complete
            current_frame_rows.append(line)

        # Process the very last frame
        if current_frame_rows:
            rectified_rows = rectify_single_frame(current_frame_rows, current_to_established_map, established_ids)
            writer.writerows(rectified_rows)

    # Display the final learned mapping for verification
    print(f"Final Persistent Mapping: {current_to_established_map}")

In [ ]:
csv_path = "/content/phua_vid_3 annotated.csv"
out_path = "/content/phua_vid_2 rectified.csv"

established_ids = discover_established_ids(csv_path, 4)
print(f"Established IDs: {sorted(list(established_ids), key=int)}")

rectify_tracks(input_csv_path=csv_path, output_csv_path=out_path, established_ids=established_ids)

Established IDs: ['1', '2', '3', '4']
Final Persistent Mapping: {'8': '1', '11': '1', '12': '1', '15': '3', '16': '4', '17': '4', '19': '4'}


# **To draw bounding boxes from a loaded CSV**

In [ ]:
def load_tracks_from_csv(csv_path: str) -> Dict[int, np.ndarray]:
    """
    Loads track data from a CSV file into a frame-indexed dictionary.

    Returns:
        {frame_idx: np.ndarray([x1,y1,x2,y2,track_id,conf,cls,ind], ...), ...}
        Note: We add dummy conf, cls, and ind columns (index 5, 6, 7) for compatibility.
    """
    frame_tracks = {}

    # Define column names based on your export_tracks_csv
    COLUMNS = ["frame", "id", "x1", "y1", "x2", "y2", "conf", "cls"]
    IDX = {name: i for i, name in enumerate(COLUMNS)}

    with open(csv_path, "r") as f:
        reader = csv.reader(f)
        header = next(reader) # Skip header

        for row in reader:
            # Convert string columns to appropriate types
            try:
                frame = int(row[IDX["frame"]])
                tid = float(row[IDX["id"]]) # Use float for ID, as the tracker array expects it
                x1 = float(row[IDX["x1"]])
                y1 = float(row[IDX["y1"]])
                x2 = float(row[IDX["x2"]])
                y2 = float(row[IDX["y2"]])
                conf = float(row[IDX["conf"]])
                cls = float(row[IDX["cls"]])
            except ValueError:
                print(f"Skipping malformed row: {row}")
                continue

            # The StrongSort tracks array is Nx [x1,y1,x2,y2,track_id,conf,cls,ind].
            # We construct a compatible array. We can use a dummy index (ind=0).
            track_row = np.array([x1, y1, x2, y2, tid, conf, cls, 0.0], dtype=np.float32)

            if frame not in frame_tracks:
                frame_tracks[frame] = []
            frame_tracks[frame].append(track_row)

    # Convert lists of arrays into single numpy arrays per frame
    for frame, tracks_list in frame_tracks.items():
        frame_tracks[frame] = np.array(tracks_list, dtype=np.float32)

    return frame_tracks

In [ ]:
def draw_tracks(frame: np.ndarray, tracks: np.ndarray, id_color_cache: dict):
    # tracks: Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
    for tb in tracks:
        x1, y1, x2, y2, tid, conf, cls, _ = tb
        x1,y1,x2,y2,tid = int(x1),int(y1),int(x2),int(y2),int(tid)
        color = id_color_cache.setdefault(tid, (37*tid%256, 17*tid%256, 93*tid%256))
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return frame

def render_video_from_csv(
    src_path: str,
    dst_path: str,
    preloaded_tracks: Dict[int, np.ndarray],
    show_pbar: bool = True,
):
    """
    Renders a video using pre-loaded track data from a CSV, skipping detection/tracking.
    """
    cap = cv2.VideoCapture(src_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {src_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w, h = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter(dst_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or None
    pbar = tqdm(total=total, unit="frame", disable=not show_pbar)

    id_color_cache = {}
    frame_idx = 0

    t0 = time.time()
    while True:
        ok, frame = cap.read()
        if not ok: break

        # --- PLUG-IN CSV DATA HERE ---
        # Get tracks for the current frame index.
        tracks = preloaded_tracks.get(frame_idx)
        # -----------------------------

        if tracks is not None and len(tracks):
            # draw using the loaded tracks
            frame = draw_tracks(frame, tracks, id_color_cache)

        out.write(frame)
        frame_idx += 1
        pbar.update(1)

    cap.release(); out.release()
    pbar.close()
    t1 = time.time()
    print(f"Saved video to: {dst_path} | frames={frame_idx} | time={t1-t0:.1f}s | ~{frame_idx/max(t1-t0,1):.1f} FPS")

In [ ]:
CSV_PATH = "/content/fyp 2025-08-19 19-58-06 rectified.csv"
INPUT_VIDEO_PATH = "/content/fyp 2025-08-19 19-58-06.mp4"
OUTPUT_VIDEO_PATH = "/content/fyp 2025-08-19 19-58-06 rectified.mp4" # New output file

# 1. Load the tweaked data
print(f"Loading tracks from: {CSV_PATH}")
tweaked_tracks = load_tracks_from_csv(CSV_PATH)

# 2. Render the video using the loaded data
render_video_from_csv(
    INPUT_VIDEO_PATH,
    OUTPUT_VIDEO_PATH,
    tweaked_tracks,
    show_pbar=True
)

Loading tracks from: /content/fyp 2025-08-19 19-58-06 rectified.csv


  0%|          | 0/317 [00:00<?, ?frame/s]

Saved video to: /content/fyp 2025-08-19 19-58-06 rectified.mp4 | frames=317 | time=7.2s | ~44.2 FPS
